In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
import shutil

shutil.make_archive('gpt-3.5-turbo_distrilBert', 'zip', root_dir='/content/processed_data/gpt-3.5-turbo-1106-distilbert')

In [2]:
!unzip processed_data.zip

Archive:  processed_data.zip
   creating: processed_data/
   creating: processed_data/claude-instant-v1/
  inflating: processed_data/claude-instant-v1/test.csv  
  inflating: processed_data/claude-instant-v1/train.csv  
   creating: processed_data/claude-v1/
  inflating: processed_data/claude-v1/test.csv  
  inflating: processed_data/claude-v1/train.csv  
   creating: processed_data/claude-v2/
  inflating: processed_data/claude-v2/test.csv  
  inflating: processed_data/claude-v2/train.csv  
   creating: processed_data/gpt-3.5-turbo-1106/
  inflating: processed_data/gpt-3.5-turbo-1106/test.csv  
  inflating: processed_data/gpt-3.5-turbo-1106/train.csv  
   creating: processed_data/gpt-4-1106-preview/
  inflating: processed_data/gpt-4-1106-preview/test.csv  
  inflating: processed_data/gpt-4-1106-preview/train.csv  
   creating: processed_data/meta-code-llama-instruct-34b-chat/
  inflating: processed_data/meta-code-llama-instruct-34b-chat/test.csv  
  inflating: processed_data/meta-code-

In [3]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313501 sha256=7fc5e5a5569cfb68feb10ecb323e901d668ba8ffac8639d44214af4b301cbe9a
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [8]:
!pip install numpy==1.26.4

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)


In [7]:
!pip uninstall numpy

Found existing installation: numpy 2.2.4
Uninstalling numpy-2.2.4:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.2.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-6bb31eeb.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? y
  Successfully uninstalled numpy-2.2.4


In [5]:

import pandas as pd
import os
import fasttext
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.linear_model import LinearRegression

# import pandas as pd
import os
import fasttext
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import re

# import fasttext
# import numpy as np

# Patch FastText to avoid NumPy 2.0 crash

# Preprocess text: lowercase + remove punctuation
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    return text


# Patch FastText model object AFTER training to fix NumPy issue
def patch_predict(model):
    original_predict = model.__class__.predict  # Get from the class, not from the instance

    def safe_predict(self, text, k=1, threshold=0.0):
        labels, probs = original_predict(self, text, k, threshold)
        return labels, np.asarray(probs)

    # Replace the method for this instance only
    import types
    model.predict = types.MethodType(safe_predict, model)



def train_and_evaluate_fasttext(train_df, test_df, model_name, output_dir='processed_data'):
    model_dir = os.path.join(output_dir, model_name.replace("/", "-"))
    os.makedirs(model_dir, exist_ok=True)

    train_file = os.path.join(model_dir, 'fasttext_train.txt')
    test_file = os.path.join(model_dir, 'fasttext_test.txt')

    # Ensure column names are usable
    if 'prompt' not in train_df.columns or model_name not in train_df.columns:
        raise ValueError(f"Missing required columns in train_df: 'prompt' and '{model_name}'")

    # Create FastText format: __label__score <text>
    train_df['fasttext_line'] = train_df.apply(
        lambda x: f"__label__{x[model_name]} {preprocess_text(x['prompt'])}", axis=1
    )
    test_df['fasttext_line'] = test_df.apply(
        lambda x: f"__label__{x[model_name]} {preprocess_text(x['prompt'])}", axis=1
    )

    train_df['fasttext_line'].to_csv(train_file, index=False, header=False, quoting=3, escapechar='\\')
    test_df['fasttext_line'].to_csv(test_file, index=False, header=False, quoting=3, escapechar='\\')

    # Train FastText model
    model = fasttext.train_supervised(
        input=train_file,
        epoch=10,
        lr=0.1,
        wordNgrams=3,
        loss='hs'
    )

    patch_predict(model)  # ✅ Fix NumPy predict crash

    # Predict helper
    def get_predictions(file_path, model):
        texts = [line.split(' ', 1)[1].strip() for line in open(file_path, encoding='utf-8')]
        true_labels = [float(line.split(' ', 1)[0].replace('__label__', '')) for line in open(file_path, encoding='utf-8')]
        pred_labels = [model.predict(text)[1][0] for text in texts]
        return true_labels, pred_labels

    # Evaluate
    train_true, train_pred = get_predictions(train_file, model)
    test_true, test_pred = get_predictions(test_file, model)

    train_mse = mean_squared_error(train_true, train_pred)
    test_mse = mean_squared_error(test_true, test_pred)
    train_mae = mean_absolute_error(train_true, train_pred)
    test_mae = mean_absolute_error(test_true, test_pred)

    acc_score = accuracy_score(train_true, train_pred)
    print(f"Accuracy Score: {acc_score}")



    print(f"\n🚀 FastText Regression Results for: {model_name}")

    print(f"Train MSE: {train_mse:.4f}, MAE: {train_mae:.4f}")
    print(f"Test  MSE: {test_mse:.4f}, MAE: {test_mae:.4f}")

    # model.save_model(os.path.join(model_dir, 'fasttext_regression_model.bin'))
    model_path = os.path.join(model_dir, 'fasttext_regression_model.bin')
    model.save_model(model_path)
    print(f"📦 FastText model saved to: {model_path}")

def train_and_evaluate_distilbert(train_df, test_df, model_name, output_dir='processed_data'):
    # Load tokenizer and model
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=1)
    model.to(device)  # Move model to GPU

    # Prepare datasets
    train_df['label'] = train_df[model_name].astype(float)
    test_df['label'] = test_df[model_name].astype(float)

    train_dataset = Dataset.from_pandas(train_df[['prompt', 'label']])
    test_dataset = Dataset.from_pandas(test_df[['prompt', 'label']])

    # Tokenize on GPU
    def tokenize_function(examples):
        return tokenizer(
            examples['prompt'],
            padding='max_length',
            truncation=True,
            max_length=512,  # Adjust based on GPU memory
            return_tensors="pt"
        )

    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Configure training for CUDA and mixed precision
    training_args = TrainingArguments(
        output_dir=f'./results/{model_name.replace("/", "-")}',
        num_train_epochs=3,
        per_device_train_batch_size=32,  # Increase batch size for GPU
        per_device_eval_batch_size=32,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        fp16=True,  # Enable mixed precision
        load_best_model_at_end=True,
        logging_dir='./logs',
        report_to="none"
    )

    # Ensure metrics are computed on CPU
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = predictions.squeeze().cpu().numpy()  # Move to CPU
        labels = labels.cpu().numpy()
        mse = mean_squared_error(labels, predictions)
        mae = mean_absolute_error(labels, predictions)
        return {"mse": mse, "mae": mae}

    # Initialize Trainer with GPU
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    # Train on GPU
    trainer.train()
    eval_results = trainer.evaluate()
    print(f"DistilBERT Regression - Test MSE: {eval_results['eval_mse']:.4f}, MAE: {eval_results['eval_mae']:.4f}")
def main(model_type="fasttext", data_dir='/content/processed_data'):
    """
    Load preprocessed data and train/evaluate the chosen regression model.
    """
    for model_name in os.listdir(data_dir):
        model_path = os.path.join(data_dir, model_name)
        if os.path.isdir(model_path):
            train_df = pd.read_csv(os.path.join(model_path, 'train.csv'))
            test_df = pd.read_csv(os.path.join(model_path, 'test.csv'))

            # Ensure the DataFrame has columns: 'prompt' (text) and model_name (scores)
            if 'prompt' not in train_df.columns or model_name not in train_df.columns:
                print(f"Skipping {model_name}: Missing 'prompt' or score column.")
                continue

            print(f"\nTraining for: {model_name}")
            if model_type == "fasttext":
                train_and_evaluate_fasttext(train_df, test_df, model_name, data_dir)
            elif model_type == "distilbert":
                train_and_evaluate_distilbert(train_df, test_df, model_name, data_dir)
            else:
                print(f"Invalid model_type: {model_type}. Choose 'fasttext' or 'distilbert'.")

if __name__ == "__main__":
    # main(model_type="fasttext")
    main(model_type="distilbert")  # Uncomment for DistilBERT



Skipping mistralai-mistral-7b-chat: Missing 'prompt' or score column.

Training for: gpt-3.5-turbo-1106

🚀 FastText Regression Results for: gpt-3.5-turbo-1106
Train MSE: 0.2215, MAE: 0.4467
Test  MSE: 0.2227, MAE: 0.4473
📦 FastText model saved to: /content/processed_data/gpt-3.5-turbo-1106/fasttext_regression_model.bin

Training for: claude-v1

🚀 FastText Regression Results for: claude-v1
Train MSE: 0.1925, MAE: 0.4043
Test  MSE: 0.2068, MAE: 0.4204
📦 FastText model saved to: /content/processed_data/claude-v1/fasttext_regression_model.bin
Skipping meta-llama-2-70b-chat: Missing 'prompt' or score column.

Training for: gpt-4-1106-preview

🚀 FastText Regression Results for: gpt-4-1106-preview
Train MSE: 0.1681, MAE: 0.3678
Test  MSE: 0.1776, MAE: 0.3781
📦 FastText model saved to: /content/processed_data/gpt-4-1106-preview/fasttext_regression_model.bin
Skipping WizardLM-WizardLM-13B-V1.2: Missing 'prompt' or score column.
Skipping zero-one-ai-Yi-34B-Chat: Missing 'prompt' or score column.

In [7]:
import fasttext
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os

def patch_fasttext_predict():
    from fasttext.FastText import _FastText
    original_predict = _FastText.predict
    def patched_predict(self, text, k=1, threshold=0.0):
        labels, probs = original_predict(self, text, k, threshold)
        return labels, np.asarray(probs)
    _FastText.predict = patched_predict

patch_fasttext_predict()  # 🛡 Apply the patch once

def evaluate_fasttext_model_accuracy(train_file, test_file, model_path, model_name="FastText"):
    model = fasttext.load_model(model_path)

    def get_predictions(file_path):
        texts = [line.split(' ', 1)[1].strip() for line in open(file_path, encoding='utf-8')]
        true_labels = [float(line.split(' ', 1)[0].replace('__label__', '')) for line in open(file_path, encoding='utf-8')]
        pred_labels = [model.predict(text)[1][0] for text in texts]
        return true_labels, pred_labels

    def compute_accuracy(true, pred, tolerance=0.5):
        rounded_acc = np.mean([round(t) == round(p) for t, p in zip(true, pred)])
        within_tolerance = np.mean([abs(t - p) <= tolerance for t, p in zip(true, pred)])
        return rounded_acc, within_tolerance

    train_true, train_pred = get_predictions(train_file)
    test_true, test_pred = get_predictions(test_file)

    train_mse = mean_squared_error(train_true, train_pred)
    test_mse = mean_squared_error(test_true, test_pred)
    train_mae = mean_absolute_error(train_true, train_pred)
    test_mae = mean_absolute_error(test_true, test_pred)

    train_acc, train_tol_acc = compute_accuracy(train_true, train_pred)
    test_acc, test_tol_acc = compute_accuracy(test_true, test_pred)

    print(f"\n✅ Evaluation for model: {model_name}")
    print(f"Train Accuracy (Rounded): {train_acc:.4f}, Within ±0.5: {train_tol_acc:.4f}")
    print(f"Test  Accuracy (Rounded): {test_acc:.4f}, Within ±0.5: {test_tol_acc:.4f}")
    print(f"Train MSE: {train_mse:.4f}, MAE: {train_mae:.4f}")
    print(f"Test  MSE: {test_mse:.4f}, MAE: {test_mae:.4f}")


In [10]:
evaluate_fasttext_model_accuracy(
    train_file='processed_data/gpt-3.5-turbo-1106/fasttext_train.txt',
    test_file='processed_data/gpt-3.5-turbo-1106/fasttext_test.txt',
    model_path='processed_data/gpt-3.5-turbo-1106/fasttext_regression_model.bin',
    model_name='gpt-3.5-turbo-1106'
)


✅ Evaluation for model: gpt-3.5-turbo-1106
Train Accuracy (Rounded): 0.5000, Within ±0.5: 0.6226
Test  Accuracy (Rounded): 0.5006, Within ±0.5: 0.6248
Train MSE: 0.2215, MAE: 0.4467
Test  MSE: 0.2227, MAE: 0.4473


In [11]:
evaluate_fasttext_model_accuracy(
    train_file='/content/processed_data/claude-instant-v1/fasttext_train.txt',
    test_file='/content/processed_data/claude-instant-v1/fasttext_test.txt',
    model_path='/content/processed_data/claude-instant-v1/fasttext_regression_model.bin',
    model_name='claude-instant-v1'
)


✅ Evaluation for model: claude-instant-v1
Train Accuracy (Rounded): 0.4744, Within ±0.5: 0.6276
Test  Accuracy (Rounded): 0.4705, Within ±0.5: 0.6277
Train MSE: 0.2209, MAE: 0.4455
Test  MSE: 0.2222, MAE: 0.4468


In [12]:
evaluate_fasttext_model_accuracy(
    train_file='/content/processed_data/gpt-4-1106-preview/fasttext_train.txt',
    test_file='/content/processed_data/gpt-4-1106-preview/fasttext_test.txt',
    model_path='/content/processed_data/gpt-4-1106-preview/fasttext_regression_model.bin',
    model_name='gpt-4-1106-preview'
)


✅ Evaluation for model: gpt-4-1106-preview
Train Accuracy (Rounded): 0.6307, Within ±0.5: 0.7829
Test  Accuracy (Rounded): 0.6193, Within ±0.5: 0.7701
Train MSE: 0.1681, MAE: 0.3678
Test  MSE: 0.1776, MAE: 0.3781


In [14]:
evaluate_fasttext_model_accuracy(
    train_file='/content/processed_data/claude-v2/fasttext_train.txt',
    test_file='/content/processed_data/claude-v2/fasttext_test.txt',
    model_path='/content/processed_data/claude-v2/fasttext_regression_model.bin',
    model_name='claude-v2'
)


✅ Evaluation for model: claude-v2
Train Accuracy (Rounded): 0.5479, Within ±0.5: 0.7222
Test  Accuracy (Rounded): 0.4895, Within ±0.5: 0.6659
Train MSE: 0.1933, MAE: 0.4127
Test  MSE: 0.2128, MAE: 0.4341


In [ ]:

import pandas as pd
import os
import fasttext
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.linear_model import LinearRegression

# import pandas as pd
import os
import fasttext
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import re

# import fasttext
# import numpy as np

# Patch FastText to avoid NumPy 2.0 crash

# Preprocess text: lowercase + remove punctuation
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    return text


# Patch FastText model object AFTER training to fix NumPy issue
def patch_predict(model):
    original_predict = model.__class__.predict  # Get from the class, not from the instance

    def safe_predict(self, text, k=1, threshold=0.0):
        labels, probs = original_predict(self, text, k, threshold)
        return labels, np.asarray(probs)

    # Replace the method for this instance only
    import types
    model.predict = types.MethodType(safe_predict, model)



def train_and_evaluate_fasttext(train_df, test_df, model_name, output_dir='processed_data'):
    model_dir = os.path.join(output_dir, model_name.replace("/", "-"))
    os.makedirs(model_dir, exist_ok=True)

    train_file = os.path.join(model_dir, 'fasttext_train.txt')
    test_file = os.path.join(model_dir, 'fasttext_test.txt')

    # Ensure column names are usable
    if 'prompt' not in train_df.columns or model_name not in train_df.columns:
        raise ValueError(f"Missing required columns in train_df: 'prompt' and '{model_name}'")

    # Create FastText format: __label__score <text>
    train_df['fasttext_line'] = train_df.apply(
        lambda x: f"__label__{x[model_name]} {preprocess_text(x['prompt'])}", axis=1
    )
    test_df['fasttext_line'] = test_df.apply(
        lambda x: f"__label__{x[model_name]} {preprocess_text(x['prompt'])}", axis=1
    )

    train_df['fasttext_line'].to_csv(train_file, index=False, header=False, quoting=3, escapechar='\\')
    test_df['fasttext_line'].to_csv(test_file, index=False, header=False, quoting=3, escapechar='\\')

    # Train FastText model
    model = fasttext.train_supervised(
        input=train_file,
        epoch=10,
        lr=0.1,
        wordNgrams=3,
        loss='hs'
    )

    patch_predict(model)  # ✅ Fix NumPy predict crash

    # Predict helper
    def get_predictions(file_path, model):
        texts = [line.split(' ', 1)[1].strip() for line in open(file_path, encoding='utf-8')]
        true_labels = [float(line.split(' ', 1)[0].replace('__label__', '')) for line in open(file_path, encoding='utf-8')]
        pred_labels = [model.predict(text)[1][0] for text in texts]
        return true_labels, pred_labels

    # Evaluate
    train_true, train_pred = get_predictions(train_file, model)
    test_true, test_pred = get_predictions(test_file, model)

    train_mse = mean_squared_error(train_true, train_pred)
    test_mse = mean_squared_error(test_true, test_pred)
    train_mae = mean_absolute_error(train_true, train_pred)
    test_mae = mean_absolute_error(test_true, test_pred)

    acc_score = accuracy_score(train_true, train_pred)
    print(f"Accuracy Score: {acc_score}")



    print(f"\n🚀 FastText Regression Results for: {model_name}")

    print(f"Train MSE: {train_mse:.4f}, MAE: {train_mae:.4f}")
    print(f"Test  MSE: {test_mse:.4f}, MAE: {test_mae:.4f}")

    # model.save_model(os.path.join(model_dir, 'fasttext_regression_model.bin'))
    model_path = os.path.join(model_dir, 'fasttext_regression_model.bin')
    model.save_model(model_path)
    print(f"📦 FastText model saved to: {model_path}")

def train_and_evaluate_distilbert(train_df, test_df, model_name, output_dir='processed_data'):
    from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

    # Load tokenizer and model
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=1)
    model.to(device)

    # Prepare datasets
    train_df['label'] = train_df[model_name].astype(float)
    test_df['label'] = test_df[model_name].astype(float)

    train_dataset = Dataset.from_pandas(train_df[['prompt', 'label']])
    test_dataset = Dataset.from_pandas(test_df[['prompt', 'label']])

    def tokenize_function(examples):
        return tokenizer(
            examples['prompt'],
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Define model output directory
    model_output_dir = os.path.join(output_dir, model_name.replace("/", "-") + "-distilbert")
    os.makedirs(model_output_dir, exist_ok=True)

    training_args = TrainingArguments(
        output_dir=model_output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        fp16=True,
        load_best_model_at_end=True,
        logging_dir=os.path.join(model_output_dir, "logs"),
        report_to="none"
    )

    def compute_metrics(eval_pred):
      predictions, labels = eval_pred

      # Ensure both are NumPy arrays
      if hasattr(predictions, "cpu"):
          predictions = predictions.cpu().numpy()
      if hasattr(labels, "cpu"):
          labels = labels.cpu().numpy()

      predictions = predictions.squeeze()

      mse = mean_squared_error(labels, predictions)
      mae = mean_absolute_error(labels, predictions)
      return {"mse": mse, "mae": mae}


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_results = trainer.evaluate()

    print(f"DistilBERT Regression - Test MSE: {eval_results['eval_mse']:.4f}, MAE: {eval_results['eval_mae']:.4f}")

    # ✅ Save model to unique folder
    trainer.save_model(model_output_dir)
    tokenizer.save_pretrained(model_output_dir)
    print(f"📦 DistilBERT model saved to: {model_output_dir}")


if __name__ == "__main__":
    # main(model_type="fasttext")
    main(model_type="distilbert")  # Uncomment for DistilBERT



Skipping mistralai-mistral-7b-chat: Missing 'prompt' or score column.

Training for: gpt-3.5-turbo-1106


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/49996 [00:00<?, ? examples/s]

Map:   0%|          | 0/21428 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/49996 [00:00<?, ? examples/s]

Map:   0%|          | 0/21428 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Mse,Mae
1,0.165300,0.164238,0.164238,0.335810


In [ ]:
import os
import torch
import fasttext
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Set device to GPU if available (for DistilBERT)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)} for DistilBERT")
else:
    print("No GPU available, DistilBERT falling back to CPU")

# Load DistilBERT model and tokenizer
def load_distilbert_model(model_dir):
    try:
        tokenizer = DistilBertTokenizer.from_pretrained(model_dir)
        model = DistilBertForSequenceClassification.from_pretrained(model_dir)
        model.to(device)
        model.eval()
        return model, tokenizer
    except Exception as e:
        print(f"Error loading DistilBERT from {model_dir}: {e}")
        return None, None

# Predict score with DistilBERT
def predict_distilbert_score(prompt, model, tokenizer):
    inputs = tokenizer(
        prompt,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    score = outputs.logits.squeeze().item()
    return max(0.0, min(1.0, score))

# Load FastText model
def load_fasttext_model(model_path):
    try:
        model = fasttext.load_model(model_path)
        return model
    except Exception as e:
        print(f"Error loading FastText from {model_path}: {e}")
        return None

# Predict score with FastText
def predict_fasttext_score(prompt, model):
    # FastText regression outputs a number directly
    score = float(model.predict([prompt])[0][0])
    return max(0.0, min(1.0, score))

# Ensemble prediction
def ensemble_score(distilbert_score, fasttext_score, weight_distilbert=0.5):
    """Combine scores with a weighted average."""
    weight_fasttext = 1.0 - weight_distilbert
    ensemble = (weight_distilbert * distilbert_score) + (weight_fasttext * fasttext_score)
    return ensemble

def test_new_prompt(prompt, data_dir='/content/processed_data', weight_distilbert=0.5):
    """Test a prompt with ensembled DistilBERT and FastText models."""
    print(f"\nTesting prompt: '{prompt}'")
    scores = {}

    for model_name in os.listdir(data_dir):
        model_path = os.path.join(data_dir, model_name)
        if not os.path.isdir(model_path):
            continue

        # Load DistilBERT
        distilbert_dir = os.path.join(model_path, "distilbert_model")
        distilbert_model, distilbert_tokenizer = None, None
        if os.path.exists(distilbert_dir) and os.path.exists(os.path.join(distilbert_dir, "pytorch_model.bin")):
            distilbert_model, distilbert_tokenizer = load_distilbert_model(distilbert_dir)

        # Load FastText
        fasttext_path = os.path.join(model_path, "fasttext_model", "model.bin")
        fasttext_model = None
        if os.path.exists(fasttext_path):
            fasttext_model = load_fasttext_model(fasttext_path)

        # Skip if neither model is available
        if distilbert_model is None and fasttext_model is None:
            print(f"No models found for {model_name}")
            continue

        # Predict scores
        distilbert_score = None
        if distilbert_model and distilbert_tokenizer:
            distilbert_score = predict_distilbert_score(prompt, distilbert_model, distilbert_tokenizer)
            print(f"{model_name} (DistilBERT): {distilbert_score:.4f}")

        fasttext_score = None
        if fasttext_model:
            fasttext_score = predict_fasttext_score(prompt, fasttext_model)
            print(f"{model_name} (FastText): {fasttext_score:.4f}")

        # Ensemble if both available, else use available one
        if distilbert_score is not None and fasttext_score is not None:
            final_score = ensemble_score(distilbert_score, fasttext_score, weight_distilbert)
            print(f"{model_name} (Ensemble): {final_score:.4f}")
            scores[model_name] = final_score
        elif distilbert_score is not None:
            print(f"{model_name} (Only DistilBERT available): {distilbert_score:.4f}")
            scores[model_name] = distilbert_score
        elif fasttext_score is not None:
            print(f"{model_name} (Only FastText available): {fasttext_score:.4f}")
            scores[model_name] = fasttext_score

    return scores

def main():
    new_prompt = "Explain relativity simply"
    scores = test_new_prompt(new_prompt, data_dir='/content/processed_data', weight_distilbert=0.5)
    if scores:
        best_llm = max(scores, key=scores.get)
        print(f"\nBest LLM for '{new_prompt}': {best_llm} with ensemble score {scores[best_llm]:.4f}")

if _name_ == "_main_":
    main